In [106]:
import numpy as np
import pandas as pd

In [107]:
data = pd.read_csv('train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [108]:
survival_data = data['Survived']
survival_data.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [109]:
data['Sex'] = data['Sex'].replace('male', 0)
data['Sex'] = data['Sex'].replace('female', 1)
data['Embarked'] = data['Embarked'].replace('S', 0)
data['Embarked'] = data['Embarked'].replace('C', 1)
data['Embarked'] = data['Embarked'].replace('Q', 2)
others = data.drop(columns=['PassengerId', 'Survived', 'Name','Ticket', 'Cabin'])
others['Age'] = others['Age'].fillna(others['Age'].median())
others['Embarked'] = others['Embarked'].fillna(others['Embarked'].mode()[0])
others = (others - others.min()) / (others.max() - others.min())
others.head()

/var/folders/6_/741c2f_d1jqgw94qpfv16v4m0000gn/T/ipykernel_8139/3681078618.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Sex'] = data['Sex'].replace('female', 1)
/var/folders/6_/741c2f_d1jqgw94qpfv16v4m0000gn/T/ipykernel_8139/3681078618.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Embarked'] = data['Embarked'].replace('Q', 2)


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1.0,0.0,0.271174,0.125,0.0,0.014151,0.0
1,0.0,1.0,0.472229,0.125,0.0,0.139136,0.5
2,1.0,1.0,0.321438,0.000,0.0,0.015469,0.0
3,0.0,1.0,0.434531,0.125,0.0,0.103644,0.0
4,1.0,0.0,0.434531,0.000,0.0,0.015713,0.0


In [110]:
train_x = np.array(others)
train_y = np.array(survival_data)
print(train_x.shape, train_y.shape)



(891, 7) (891,)


n_x = 7
n_h = 891
n_y = 1

In [111]:
sizes = [7, 16, 8, 1]
def initialize_weights(sizes):
    
    parameters = {}
    L = len(sizes)
    
    for i in range(1, L):
        parameters["W"+str(i)] = np.random.randn(sizes[i], sizes[i-1]) * np.sqrt(2 / sizes[i-1])
        parameters["b"+str(i)] = np.zeros((sizes[i], 1))
    
    return parameters
    

In [112]:
def forward(A_prev, W, b):
    Z = np.dot(W, A_prev) + b
    cache = (A_prev, W, b)
    return Z, cache

In [113]:
def linear_activation_forward(A_prev, W, b, activation):
    
    if activation == "sigmoid":
        Z, linear_cache = forward(A_prev, W, b)
        A = 1 / (1 + np.exp(-Z))
        activation_cache = Z
        
    elif activation == "relu":
        Z, linear_cache = forward(A_prev, W, b)
        Z = np.clip(Z, -500, 500)
        A = np.maximum(0, Z)
        activation_cache = Z
    
    cache = (linear_cache, activation_cache)
    return A, cache

In [114]:
def L_model_forward(X, parameters):
    
    caches = []
    A = X
    L = len(parameters) // 2
    
    for i in range(1, L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters["W"+str(i)], parameters["b"+str(i)], "relu")
        caches.append(cache)
    
    AL, cache = linear_activation_forward(A, parameters["W"+str(L)], parameters["b"+str(L)], "sigmoid")
    caches.append(cache)
    
    return AL, caches

In [115]:
def compute_cost(AL, Y):
    m = Y.shape[0]
    cost = - (1/m) * np.sum(Y * np.log(AL) + (1 - Y) * np.log(1 - AL))
    cost = np.squeeze(cost)
    return cost

In [116]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    dW = (1/m) * np.dot(dZ, A_prev.T)
    db = (1/m) * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)
    
    return dA_prev, dW, db

In [117]:
def linear_activation_backward(dA, cache, activation):
    
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        Z = activation_cache
        dZ = np.array(dA, copy=True)
        dZ[Z <= 0] = 0
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    elif activation == "sigmoid":
        Z = activation_cache
        s = 1 / (1 + np.exp(-Z))
        dZ = dA * s * (1 - s)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db

In [118]:
def L_model_backward(AL, Y, caches):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    current_cache = caches[L-1]
    grads["dA"+str(L-1)], grads["dW"+str(L)], grads["db"+str(L)] = linear_activation_backward(dAL, current_cache, "sigmoid")
    
    for i in reversed(range(L-1)):
        current_cache = caches[i]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA"+str(i+1)], current_cache, "relu")
        grads["dA"+str(i)] = dA_prev_temp
        grads["dW"+str(i+1)] = dW_temp
        grads["db"+str(i+1)] = db_temp
    
    return grads

In [119]:
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2
    
    for i in range(1, L+1):
        parameters["W"+str(i)] = parameters["W"+str(i)] - learning_rate * grads["dW"+str(i)]
        parameters["b"+str(i)] = parameters["b"+str(i)] - learning_rate * grads["db"+str(i)]
    
    return parameters

In [120]:
def L_model(X, Y, sizes, learning_rate = 0.0075, num_iterations = 3000, print_cost = False):
    
    costs = []
    parameters = initialize_weights(sizes)
    
    for i in range(0, num_iterations):
        
        AL, caches = L_model_forward(X, parameters)
        
        cost = compute_cost(AL, Y)
        
        grads = L_model_backward(AL, Y, caches)
        
        parameters = update_parameters(parameters, grads, learning_rate)
        
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
            costs.append(cost)
    
    return parameters, costs

In [121]:
parameters, costs = L_model(train_x.T, train_y, sizes, learning_rate=0.01, num_iterations=5000, print_cost=True)

print(costs)

Cost after iteration 0: 0.702855
Cost after iteration 100: 0.605836
Cost after iteration 200: 0.557379
Cost after iteration 300: 0.527448
Cost after iteration 400: 0.507528
Cost after iteration 500: 0.493902
Cost after iteration 600: 0.484124
Cost after iteration 700: 0.476945
Cost after iteration 800: 0.471493
Cost after iteration 900: 0.467209
Cost after iteration 1000: 0.463634
Cost after iteration 1100: 0.460654
Cost after iteration 1200: 0.458098
Cost after iteration 1300: 0.455909
Cost after iteration 1400: 0.454011
Cost after iteration 1500: 0.452338
Cost after iteration 1600: 0.450723
Cost after iteration 1700: 0.449086
Cost after iteration 1800: 0.447641
Cost after iteration 1900: 0.446276
Cost after iteration 2000: 0.444961
Cost after iteration 2100: 0.443714
Cost after iteration 2200: 0.442530
Cost after iteration 2300: 0.441409
Cost after iteration 2400: 0.440337
Cost after iteration 2500: 0.439338
Cost after iteration 2600: 0.438406
Cost after iteration 2700: 0.437516
Cost

In [122]:
def predict(X, parameters):
    AL, _ = L_model_forward(X, parameters)
    
    predictions = (AL > 0.5).astype(int)
    
    return predictions

In [124]:
print(predict(train_x.T, parameters))

accuracy = np.mean(predict(train_x.T, parameters) == train_y.reshape(1, -1)) * 100
print(f'Accuracy: {accuracy}%')

[[0 1 1 1 0 0 0 0 0 1 1 1 0 0 1 1 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0 1 1 0 0 0
  0 0 0 1 0 1 0 1 1 0 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0
  0 0 0 0 0 0 1 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0
  0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 1 0 0
  0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0
  0 0 0 0 1 0 1 0 0 0 1 0 1 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1
  1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 1 0 0 0 0
  0 0 0 1 1 1 1 1 0 0 0 0 1 0 0 0 1 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
  0 1 1 1 0 1 0 1 0 1 0 1 1 0 0 1 0 1 1 1 0 1 1 1 1 0 0 1 1 0 1 1 0 0 1 1
  0 1 0 1 0 1 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1
  0 0 1 0 0 0 1 1 1 1 1 0 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 1 0 0
  1 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0
  1 0 0 1 0 1 0 0 1 0 0 1 0 1 1 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0
  0 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 

In [ ]:
test_data = pd.read_csv('test.csv')
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [138]:
test_others = test_data.drop(columns=['PassengerId', 'Name','Ticket', 'Cabin'])
test_others['Sex'] = test_data['Sex'].replace('male', 0)
test_others['Sex'] = test_data['Sex'].replace('female', 1)
test_others['Embarked'] = test_data['Embarked'].replace('S', 0)
test_others['Embarked'] = test_data['Embarked'].replace('C', 1)
test_others['Embarked'] = test_data['Embarked'].replace('Q', 2)
test_others['Age'] = test_others['Age'].fillna(test_others['Age'].median())
test_others['Fare'] = test_others['Fare'].fillna(test_others['Fare'].median())
test_others['Embarked'] = test_others['Embarked'].fillna(test_others['Embarked'].mode()[0])
test_others = (test_others - test_others.min()) / (test_others.max() - test_others.min())

test_x = np.array(test_others)


In [139]:
test_survival = pd.read_csv('gender_submission.csv')['Survived']
test_y = np.array(test_survival)
print(predict(test_x.T, parameters))
accuracy = np.mean(predict(test_x.T, parameters) == test_y.reshape(1, -1)) * 100
print(f'Accuracy: {accuracy}%')


[[0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 0 0 1 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0
  1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0
  1 1 1 0 0 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0
  0 0 0 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0
  0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 1 1
  0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1 1 0 1 0 0 0 0 1 0 0
  1 0 1 0 1 0 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0
  0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 0 0
  0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0
  1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 0
  0 1 1 0 1 0 0 1 1 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1
  0 1 0 0 1 0 1 1 0 0 0 0 1 1 1 1 1 0 1 0 0 0]]
Accuracy: 94.25837320574163%
